# Naives Bayes from scratch with an self implemented roc curve
**here are the used datasets:**
    
iris: https://archive.ics.uci.edu/ml/datasets/congressional+voting+records

In [ ]:
import numpy
import pandas

In [2]:
# I decided to use gaussian naïve bayes because the data is numerical, 
# so I need something to compute the likelihood of each feature

In [3]:
data = pandas.read_csv("./iris.tmls", skiprows=[1]).rename(columns={"class": "c"})

In [4]:
d2 = data.query('c == "Iris-setosa" | c == "Iris-versicolor"')
d2

,sepal length,sepal width,petal length,petal width,c
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,Iris-versicolor
96,5.7,2.9,4.2,1.3,Iris-versicolor
97,6.2,2.9,4.3,1.3,Iris-versicolor
98,5.1,2.5,3.0,1.1,Iris-versicolor


In [5]:
data

,sepal length,sepal width,petal length,petal width,c
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [6]:
def naives_bayes_training(training_data, class1, class2):
    
    data_class1 = training_data.query(f"c == '{class1}'")
    data_class2 = training_data.query(f"c == '{class2}'")

    # calculate for each features: means, standev for both classes
    s1 = data_class1[["sepal length", "sepal width", "petal length", "petal width"]].std()
    m1 = data_class1[["sepal length", "sepal width", "petal length", "petal width"]].mean()
    
    s2 = data_class2[["sepal length", "sepal width", "petal length", "petal width"]].std()
    m2 = data_class2[["sepal length", "sepal width", "petal length", "petal width"]].mean()
    
    # compute frequency of both classes
    data1 = training_data.query(f"c == '{class1}'")
    data2 = training_data.query(f"c == '{class2}'")

    stats = {
        "s1": s1.values,
        "s2": s2.values,
        "m1": m1.values,
        "m2": m2.values,
        "f1": data1.shape[0] / training_data.shape[0],
        "f2": data2.shape[0] / training_data.shape[0],
    }
    return stats

def naives_bayes(ordered_data, class1, class2, k = 10):
    
    ordered_data = ordered_data.query(f"c == '{class1}' | c == '{class2}'")
    
    # shuffle data because the data is ordered by target class
    data = ordered_data.sample(frac=1).reset_index(drop=True)
    
    (line_nb,_) = data.shape
    
    # variable used for roc curve
    predicted_data = pandas.DataFrame(columns=["pred1", "pred2", "y"])
    
    # k cross validation    
    for i2 in range(k, line_nb + k, k):
        training_data = data.iloc[[(i2 <= i or i < i2 - k) for i in range(0, line_nb)]]
        test_data = data.iloc[[(i2 - k <= i and i < i2) for i in range(0, line_nb)]]
        stats = naives_bayes_training(training_data, class1, class2)
        
        # retrieve accuracy of this fold and compute probility and the true class
        predictions, acc = naives_bayes_test(test_data, stats, class1, class2)
        print("acc = ", acc)
        predicted_data = pandas.concat([predicted_data, predictions])
    # draw roc curve
    roc_curve(predicted_data)

In [7]:
# testing

def argmax(row):
    if row["pred1"] > row["pred2"]:
        return 1
    return 0

def translate_class(row, class1, class2):
    if row["c"] == class1:
        return 1
    return 0
    

def norm_fct(x, sigma, mu):
    return 1/(sigma * numpy.sqrt(2*numpy.pi)) * numpy.exp(-1/2* ((x - mu)/sigma)**2)

def get_likelihood(test_data, std, mean, prior):
    # actually this function return likelihood * prior (class prob)
    data = test_data[["sepal length", "sepal width", "petal length", "petal width"]].to_numpy()
    result = norm_fct(data, std, mean)
    
    return prior * numpy.prod(result, axis=1)

def naives_bayes_test(test_data, stats, class1, class2):
    # compute the posterior probability P(class|data)
    l1 = get_likelihood(test_data, stats['s1'], stats['m1'], stats['f1'])
    l2 = get_likelihood(test_data, stats['s2'], stats['m2'], stats['f2'])
    
    evidence = l1 + l2
    
    r1 = l1 / evidence
    r2 = l2 / evidence
    
    test_data.insert(0, "pred1", r1)
    test_data.insert(0, "pred2", r2)
    
    # compare prob's classes to give prediction
    tmppred = test_data.apply(lambda row: argmax(row), axis=1)
    test_data.insert(0, "pred", tmppred)
    
    # rename
    tmpy = test_data.apply(lambda row: translate_class(row, class1, class2), axis=1)
    test_data.insert(0, "y", tmpy)
    
    # calculate accuracy
    arr = test_data.apply(lambda row: row["pred"] == row["y"], axis=1)
    count_arr = numpy.bincount(arr)
    acc = (count_arr[1] / test_data.shape[0])
    
    return test_data[["pred1", "pred2", "y"]], acc

In [12]:
import plotly.express as px
import plotly.graph_objects as go

from sklearn import metrics


def get_intervals(fpr, tpr):
    indexes = [0]
    values = [fpr[0]]
    values2 = [tpr[0]]
    
    for i in range(0, len(fpr)):
        if not fpr[i] in values:
            indexes.append(i)
            values.append(fpr[i])
            values2.append(tpr[i])
        if fpr[i] in values and tpr[i] > values2[-1]:
            indexes[-1] = i
            values[-1] = fpr[i]
            values2[-1] = tpr[i]
            
    print(values)
    return indexes

def auc(tpr, fpr):
    # reverse list
    acc = 0
    print(metrics.auc(fpr, tpr))
    print(fpr)
    tpr.reverse()
    fpr.reverse()
    
    indexes = get_intervals(fpr, tpr)

    for i in range(0, len(indexes)):
        if i == len(indexes) - 1:
            break
        i1 = indexes[i]
        i2 = indexes[i + 1]
        print(f"i1  ={fpr[i1]} i2 = {fpr[i2]}")
        
        area = 0

        print(f"x1 = {fpr[i1]}, y1  = {tpr[i1]}, x2 = {fpr[i2]}, y2 = {tpr[i2]}")
        coef = (tpr[i2] - tpr[i1]) / (fpr[i2] - fpr[i1])
        print(f"coef {coef}")
        
        if coef == 0:
            area = tpr[i1] * fpr[i2] - tpr[i1] * fpr[i1]
        else:
            y2 = coef * fpr[i1]
            b = tpr[i1] - y2
            
            area = (((fpr[i2]**2)/2 * coef) + b * fpr[i2]) - (((fpr[i1]**2)/2 * coef) + b * fpr[i1])
        print(area)
        acc += area
    print(f"acc = {acc}")
    return acc
    

def draw_roc_curve(fpr, tpr):
    d = {'fpr': fpr, 'tpr': tpr}
    df = pandas.DataFrame(data=d)
    
    fig = px.line(df, x="fpr", y="tpr", title=f'ROC Curve (AUC={auc(tpr, fpr)})')

    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
    )

    fig.show()
    
def compute_rate(predictions):
    
    p = predictions.query("y == 1").shape[0]
    n = predictions.query("y == 0").shape[0]
    
    tparr = predictions.apply(lambda row: row["y"] == 1 and row["pred"] == 1, axis=1)
    fparr = predictions.apply(lambda row: row["y"] == 0 and row["pred"] == 1, axis=1)
    
    count_arr = numpy.bincount(tparr)
    tp = 0
    if (count_arr.size != 1):
        tp = count_arr[1]
    
    count_arr = numpy.bincount(fparr)
    fp = 0
    if (count_arr.size != 1):
        fp = count_arr[1]

    tpr = tp / p
    fpr = fp / n
    
    return fpr, tpr

def compare_threshold(row, threshold):
    if row["pred1"] > threshold:
        return 1
    return 0

def roc_curve(predictions):
    arr_fpr = []
    arr_tpr = []
    
    
    nbpoint = 300
    
    for t in range(0, nbpoint):
        pred = predictions.apply(lambda row: compare_threshold(row, t/nbpoint), axis=1)
        predictions.insert(0, "pred", pred)
        fpr, tpr = compute_rate(predictions)
        arr_fpr.append(fpr)
        arr_tpr.append(tpr)
        predictions = predictions.drop(['pred'], axis=1)
    draw_roc_curve(arr_fpr, arr_tpr)


In [13]:
naives_bayes(data, "Iris-versicolor", "Iris-virginica")

acc =  1.0
acc =  1.0
acc =  1.0
acc =  0.9
acc =  1.0
acc =  0.9
acc =  0.8
acc =  1.0
acc =  0.8
acc =  1.0
0.9791999999999998
[1.0, 0.3, 0.28, 0.28, 0.28, 0.24, 0.24, 0.24, 0.22, 0.22, 0.2, 0.2, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08

In [10]:
naives_bayes(data, "Iris-setosa", "Iris-virginica")

acc =  1.0
acc =  1.0
acc =  1.0
acc =  1.0
acc =  1.0
acc =  1.0
acc =  1.0
acc =  1.0
acc =  1.0
acc =  1.0
1.0
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [11]:
naives_bayes(data, "Iris-versicolor", "Iris-setosa")

acc =  1.0
acc =  1.0
acc =  1.0
acc =  1.0
acc =  1.0
acc =  1.0
acc =  1.0
acc =  1.0
acc =  1.0
acc =  1.0
1.0
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 